## 6.3. Consultas Dinâmicas

_Consultas dinâmicas_ permitem a exploração rápida e reversível de dados para isolar padrões de interesse. Conforme definido por [Ahlberg, Williamson, &amp; Shneiderman](https://www.cs.umd.edu/~ben/papers/Ahlberg1992Dynamic.pdf) uma consulta dinâmica:

- representa uma consulta graficamente,
- fornece limites visíveis no intervalo de consulta,
- fornece uma representação gráfica dos dados e do resultado da consulta,
- fornece feedback imediato do resultado após cada ajuste da consulta,
- e permite que usuários novatos comecem a trabalhar com pouco treinamento.

Uma abordagem comum é manipular parâmetros de consulta usando `widgets` de interface de usuário padrão, como controles deslizantes, botões de opção e menus suspensos. Para gerar widgets de consulta dinâmica, podemos aplicar uma operação `bind (vincular)` de seleção a um ou mais campos de dados que desejamos consultar.

Vamos construir um gráfico de dispersão interativo que usa uma consulta dinâmica para filtrar a exibição. Dado um gráfico de dispersão de classificações de filmes (do Rotten Tomates e IMDB), podemos adicionar uma seleção no campo `Major_Genre` para permitir a filtragem interativa por gênero de filme.

Para começar, vamos extrair os gêneros únicos (não nulos) dos dados de `movies (filmes)`:

In [25]:
df = pd.read_json(movies) # carrega dados do filme
genres = df['Major_Genre'].unique() # obtém valores de campo exclusivos
genres = list(filter(lambda d: d is not None, genres)) # filtrar valores Nenhum
genres.sort() # classificar em ordem alfabética

Para uso posterior, vamos também definir uma lista de valores `MPAA_Rating` exclusivos:

In [26]:
mpaa = ['G', 'PG', 'PG-13', 'R', 'NC-17', 'Not Rated']

Agora vamos criar uma seleção `single (única)` vinculada a um menu suspenso.

*Use o menu de consulta dinâmica abaixo para explorar os dados. Como as classificações variam por gênero? Como você revisaria o código para filtrar `MPAA_Rating` (G, PG, PG-13, etc.) em vez de `Major_Genre`?*

In [ ]:
selectGenre = alt.selection_single(
    name='Select', # name the selection 'Select'
    fields=['Major_Genre'], # limit selection to the Major_Genre field
    init={'Major_Genre': genres[0]}, # use first genre entry as initial value
    bind=alt.binding_select(options=genres) # bind to a menu of unique genre values
)

alt.Chart(movies).mark_circle().add_selection(
    selectGenre
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selectGenre, alt.value(0.75), alt.value(0.05))
)

Nossa construção acima aproveita vários aspectos das seleções:

- Damos um nome à seleção (`'Select'`). Este nome não é obrigatório, mas permite influenciar o texto do rótulo do menu de consulta dinâmica gerado. (_O que acontece se você remover o nome? Experimente!_)
- Restringimos a seleção a um campo de dados específico (`Major_Genre`). Anteriormente, quando usávamos uma seleção `single (única)`, a seleção era mapeada para pontos de dados individuais. Ao limitar a seleção a um campo específico, podemos selecionar _all (todos)_ os pontos de dados cujo valor do campo `Major_Genre` corresponde ao único valor selecionado.
- Inicializamos `init=...` a seleção com um valor inicial.
- Nós `bind (vinculamos)` a seleção a um widget de interface, neste caso um menu suspenso via `binding_select`.
- Como antes, usamos uma codificação condicional para controlar a opacidade do canal.





### 6.3.1. Vinculando seleções a múltiplas entradas

Uma instância de seleção pode ser vinculada a _múltiplos_ widgets de consulta dinâmica. Vamos modificar o exemplo acima para fornecer filtros para _ambos_ `Major_Genre` e `MPAA_Rating`, usando botões de opção em vez de um menu. Nossa seleção `single (única)` agora é definida em um único _par_ de gênero e valores de classificação MPAA

_Procure conjunções surpreendentes de gênero e classificação. Existe algum filme de terror com classificação G ou PG?_





In [ ]:
# seleção de valor único em pares [Major_Genre, MPAA_Rating]
# usa valores específicos conectados como valores iniciais selecionados
selection = alt.selection_single(
    name='Select',
    fields=['Major_Genre', 'MPAA_Rating'],
    init={'Major_Genre': 'Drama', 'MPAA_Rating': 'R'},
    bind={'Major_Genre': alt.binding_select(options=genres), 'MPAA_Rating': alt.binding_radio(options=mpaa)}
)
  
# gráfico de dispersão, modifique a opacidade com base na seleção
alt.Chart(movies).mark_circle().add_selection(
    selection
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selection, alt.value(0.75), alt.value(0.05))
)

_Curiosidades: a classificação PG-13 não existia quando os filmes [Jaws](https://www.imdb.com/title/tt0073195/) e [Jaws 2](https://www.imdb.com/title/tt0077766/) foram liberados. O primeiro filme a receber classificação PG-13 foi de 1984 [Red Dawn](https://www.imdb.com/title/tt0087985/)._

### 6.3.2. Usando visualizações como consultas dinâmicas

Embora os widgets de interface padrão mostrem os _possíveis_ valores dos parâmetros de consulta, eles não visualizam a _distribuição_ desses valores. Também podemos querer usar interações mais ricas, como seleções de múltiplos valores ou intervalos, em vez de widgets de entrada que selecionam apenas um único valor por vez.

Para resolver esses problemas, podemos criar gráficos adicionais para visualizar dados e oferecer suporte a consultas dinâmicas. Vamos adicionar um histograma da contagem de filmes por ano e usar uma seleção de intervalo para destacar dinamicamente os filmes em períodos de tempo selecionados.

*Interaja com o histograma anual para explorar filmes de diferentes períodos. Você viu alguma evidência de [viés de amostragem](https://en.wikipedia.org/wiki/Sampling_bias) ao longo dos anos? (Como as avaliações do ano e dos críticos se relacionam?)*

_Os anos vão de 1930 a 2040! Os filmes futuros estão em pré-produção ou existem erros de “um século”? Além disso, dependendo do fuso horário em que você está, você poderá ver um pequeno aumento em 1969 ou 1970. Por que isso aconteceria? (Veja o final do caderno para explicação!)_




In [ ]:
brush = alt.selection_interval(
    encodings=['x'] # limita a seleção aos valores do eixo x (ano)
)

# histograma de consulta dinâmica
years = alt.Chart(movies).mark_bar().add_selection(
    brush
).encode(
    alt.X('year(Release_Date):T', title='Films by Release Year'),
    alt.Y('count():Q', title=None)
).properties(
    width=650,
    height=50
)

# gráfico de dispersão, modifique a opacidade com base na seleção
ratings = alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(
    width=650,
    height=400
)

alt.vconcat(years, ratings).properties(spacing=5)

O exemplo acima fornece consultas dinâmicas usando uma _seleção vinculada_ entre gráficos:

- Criamos uma seleção de `intervalo` (`brush`) e definimos `encodings=['x']` para limitar a seleção apenas ao eixo x, resultando em um intervalo de seleção unidimensional.
- Registramos `brush` com nosso histograma de filmes por ano via `.add_selection(brush)`.
- Usamos `brush` em uma codificação condicional para ajustar a `opacidade` do gráfico de dispersão.

Esta técnica de interação de selecionar elementos em um gráfico e ver destaques vinculados em um ou mais gráficos é conhecida como [_brushing &amp; linking_](https://en.wikipedia.org/wiki/Brushing_and_linking).

